In [1]:
import mlflow
import mlflow.sklearn
from imblearn.over_sampling import SMOTE,ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,cross_val_predict,StratifiedKFold
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
tracking_uri = "http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000" 

In [3]:
df  = pd.read_csv("data/preprocessing_df.csv")
df

,Unnamed: 0,text,category,word_count,num_chars,num_punctuation_chars
0,0,family mormon never tried explain still stare ...,1,39,259,0
1,1,buddhism much lot compatible christianity espe...,1,196,1268,0
2,2,seriously say thing first get complex explain ...,-1,86,459,0
3,3,learned want teach different focus goal wrappi...,0,29,167,0
4,4,benefit may want read living buddha living chr...,1,112,690,0
...,...,...,...,...,...,...
199697,162975,crore paid neerav modi recovered congress lead...,-1,18,107,0
199698,162976,dear rss terrorist payal gawar modi killing pl...,-1,36,248,0
199699,162977,cover interaction forum left,0,9,50,0
199700,162978,big project came india modi dream project happ...,0,13,77,0


In [4]:
df.text = df.text.fillna("")

In [5]:
mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment("Exp4 : Handling Imbalanced Data")

2025/06/28 18:05:55 INFO mlflow.tracking.fluent: Experiment with name 'Exp4 : Handling Imbalanced Data' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-bucket-114/678493427717689127', creation_time=1751123155231, experiment_id='678493427717689127', last_update_time=1751123155231, lifecycle_stage='active', name='Exp4 : Handling Imbalanced Data', tags={}>

In [9]:
def run_imbalanced_experiment(imbalance_method):
    ngram_range = (1, 3)  # Trigram setting
    max_features = 8000
    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X = df.text
    y = df.category
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=114, stratify=y)

    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    if imbalance_method == "class_weights":
        class_weight = "balanced"
    else:
        class_weight = None
        
        if imbalance_method == "oversampling":
            smote = SMOTE(random_state=114)
            X_train_vec ,y_train = smote.fit_resample(X_train_vec,y_train)
        elif imbalance_method == "adasyn":
            adasyn = ADASYN(random_state=114)
            X_train_vec ,y_train = adasyn.fit_resample(X_train_vec,y_train)
        elif imbalance_method == "undersampling":
            rus = RandomUnderSampler(random_state=114)
            X_train_vec ,y_train = rus.fit_resample(X_train_vec,y_train)
        elif imbalance_method == "smote_enn":
            smote_enn = SMOTEENN(random_state=114)
            X_train_vec ,y_train = smote_enn.fit_resample(X_train_vec,y_train)
        
    
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "imbalance_handling")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, imbalance handling method={imbalance_method}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("imbalance_method", imbalance_method)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42, class_weight=class_weight)
        model.fit(X_train_vec, y_train)

        # Step 6: Make predictions and log metrics
        y_pred = model.predict(X_test_vec)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, Imbalance={imbalance_method}")
        confusion_matrix_filename = f"confusion_matrix_{imbalance_method}.png"
        plt.savefig(confusion_matrix_filename)
        mlflow.log_artifact(confusion_matrix_filename)
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_imbalance_{imbalance_method}")

# Step 7: Run experiments for different imbalance methods
imbalance_methods = ['class_weights', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']

for method in imbalance_methods:
    run_imbalanced_experiment(method)

       

2025/06/28 18:09:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 18:09:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_class_weights_RandomForest_TFIDF_Trigrams at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/678493427717689127/runs/091281d2fc9a4636b3b8f7bda540a70a
🧪 View experiment at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/678493427717689127


2025/06/28 18:11:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 18:11:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_oversampling_RandomForest_TFIDF_Trigrams at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/678493427717689127/runs/51bfd11a3b5b4cfd852bc330dc3922a1
🧪 View experiment at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/678493427717689127


2025/06/28 18:17:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 18:18:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_adasyn_RandomForest_TFIDF_Trigrams at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/678493427717689127/runs/135ea1cddaa1406086259ee18ef1402f
🧪 View experiment at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/678493427717689127


2025/06/28 18:18:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 18:18:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_undersampling_RandomForest_TFIDF_Trigrams at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/678493427717689127/runs/277d55cf0ae74bc2a8a907fbea11dbdf
🧪 View experiment at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/678493427717689127


2025/06/28 18:34:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 18:34:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_smote_enn_RandomForest_TFIDF_Trigrams at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/678493427717689127/runs/93fc354f76fb4e3ca97880d81edc1604
🧪 View experiment at: http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000/#/experiments/678493427717689127
